In [9]:
import itk

# Input and output file paths for 2D PNG images
input_file_path = "output_image.png"  # Replace with the actual path
real_file_path = "path_to_real_output.png"  # Replace with the actual path
imaginary_file_path = "path_to_imaginary_output.png"  # Replace with the actual path
modulus_file_path = "path_to_modulus_output.png"  # Replace with the actual path

# Define the dimension and pixel types for 2D images
Dimension = 2
FloatPixelType = itk.F  # Float type pixel
UnsignedCharPixelType = itk.UC  # Unsigned char for output
FloatImageType = itk.Image[FloatPixelType, Dimension]
UnsignedCharImageType = itk.Image[UnsignedCharPixelType, Dimension]

# Read the input image (2D)
input_image = itk.imread(input_file_path)

# Padding the image to make its size compatible with FFT
PadFilterType = itk.WrapPadImageFilter.New(input_image)
pad_filter = PadFilterType
pad_filter.SetInput(input_image)

# Define the padding values (adjust as needed for your image dimensions)
padding = [0, 0]  # Adjust padding if required
pad_filter.SetPadUpperBound(padding)
pad_filter.Update()

# Perform FFT
FFTType = itk.ForwardFFTImageFilter[FloatImageType]
fft_filter = FFTType.New()
fft_filter.SetInput(pad_filter.GetOutput())
fft_filter.Update()

FFTOutputImageType = FFTType.OutputImageType

# Extract and save the real part of the FFT
RealFilterType = itk.ComplexToRealImageFilter[FFTOutputImageType, FloatImageType]
real_filter = RealFilterType.New()
real_filter.SetInput(fft_filter.GetOutput())
real_filter.Update()

real_rescale_filter = itk.RescaleIntensityImageFilter.New(real_filter.GetOutput())
real_rescale_filter.SetOutputMinimum(itk.NumericTraits[UnsignedCharPixelType].min())
real_rescale_filter.SetOutputMaximum(itk.NumericTraits[UnsignedCharPixelType].max())
real_rescale_filter.Update()

itk.imwrite(real_rescale_filter.GetOutput(), real_file_path)

# Extract and save the imaginary part of the FFT
ImaginaryFilterType = itk.ComplexToImaginaryImageFilter[FFTOutputImageType, FloatImageType]
imaginary_filter = ImaginaryFilterType.New()
imaginary_filter.SetInput(fft_filter.GetOutput())
imaginary_filter.Update()

imaginary_rescale_filter = itk.RescaleIntensityImageFilter.New(imaginary_filter.GetOutput())
imaginary_rescale_filter.SetOutputMinimum(itk.NumericTraits[UnsignedCharPixelType].min())
imaginary_rescale_filter.SetOutputMaximum(itk.NumericTraits[UnsignedCharPixelType].max())
imaginary_rescale_filter.Update()

itk.imwrite(imaginary_rescale_filter.GetOutput(), imaginary_file_path)

# Compute and save the modulus (magnitude) of the FFT
ModulusFilterType = itk.ComplexToModulusImageFilter[FFTOutputImageType, FloatImageType]
modulus_filter = ModulusFilterType.New()
modulus_filter.SetInput(fft_filter.GetOutput())
modulus_filter.Update()

modulus_rescale_filter = itk.RescaleIntensityImageFilter.New(modulus_filter.GetOutput())
modulus_rescale_filter.SetOutputMinimum(itk.NumericTraits[UnsignedCharPixelType].min())
modulus_rescale_filter.SetOutputMaximum(itk.NumericTraits[UnsignedCharPixelType].max())
modulus_rescale_filter.Update()

itk.imwrite(modulus_rescale_filter.GetOutput(), modulus_file_path)


TemplateTypeError: itk.ForwardFFTImageFilter is not wrapped for input type `itk.Image[itk.F,2]`.

To limit the size of the package, only a limited number of
types are available in ITK Python. To print the supported
types, run the following command in your python environment:

    itk.ForwardFFTImageFilter.GetTypes()

Possible solutions:
* If you are an application user:
** Convert your input image into a supported format (see below).
** Contact developer to report the issue.
* If you are an application developer, force input images to be
loaded in a supported pixel type.

    e.g.: instance = itk.ForwardFFTImageFilter[itk.Image[itk.F,2], itk.Image[itk.complex[itk.F],2]].New(my_input)

* (Advanced) If you are an application developer, build ITK Python yourself and
turned to `ON` the corresponding CMake option to wrap the pixel type or image
dimension you need. When configuring ITK with CMake, you can set
`ITK_WRAP_${type}` (replace ${type} with appropriate pixel type such as
`double`). 